In [6]:
!conda install -c conda-forge folium=0.5.0 --yes
!conda install -c conda-forge Geopy --yes
!conda install -c conda-forge Geocoder --yes
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import geocoder
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.model_selection import KFold

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0          conda-forge
    geopy:         2.0.0-pyh9f0ad1d_0 conda-forge


geopy-2.0.0          | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction

In [25]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_ce63b663471a407d801a3f252fe0f551 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='vvgXT5WrvghR_oaHLfFo2eMmaK-5cKd27kCFXfs8ePzB',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_ce63b663471a407d801a3f252fe0f551.get_object(Bucket='dscapstone-donotdelete-pr-prx8eyvza3usuz',Key='sf_neighborhoods.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,Neighborhood
0,Bayview
1,Bernal Heights
2,Castro
3,Chinatown
4,Crocker Amazon


In [54]:
df_sf = df_data_1

geolocator = Nominatim(user_agent="sf_explorer")

def get_geocode(address):
    try:
        return geolocator.geocode(address)
    except GeocoderTimedOut:
        return get_geocode(address)

df_r = []
    
for neigh in df_sf['Neighborhood']:
    address = '{}, San Francisco, California'.format(neigh)
    location = get_geocode(address)
    if location is not None:
        df_r.append([neigh, location.latitude, location.longitude])
    else:
        df_r.append([neigh])

In [56]:
cols = ['Neighborhood', 'Latitude', 'Longitude']
df_neigh = pd.DataFrame(df_r, columns=cols).set_index('Neighborhood')

In [57]:
# Which neighborhoods are missing latitude/longitude?
df_neigh[pd.isnull(df_neigh).any(axis=1)]

,Latitude,Longitude
Neighborhood,,
Diamond Heights,NaN,NaN
Outer Richmond,NaN,NaN
Outer Sunset,NaN,NaN
Portrero Hill,NaN,NaN
West of Twin Peaks,NaN,NaN


In [58]:
# Fill in missing data manually
dh_lat, dh_lon = 37.745764, -122.441638
ph_lat, ph_lon = 37.75716, -122.39986
wotp_lat, wotp_lon = 37.739689, -122.466942
or_lat, or_lon = 37.7798, -122.4901

df_neigh.loc['Diamond Heights'] = dh_lat, dh_lon
df_neigh.loc['Portrero Hill'] = ph_lat, ph_lon
df_neigh.loc['West of Twin Peaks'] = wotp_lat, wotp_lon
df_neigh.loc['Outer Richmond'] = or_lat, or_lon

In [63]:
# Nominatim's lat/lon for Outer Sunset and Lakeshore were pretty off, so I'm correcting them here.
os_lat, os_lon = 37.753427, -122.495402
ls_lat, ls_lon = 37.723698, -122.480287

df_neigh.loc['Outer Sunset'] = os_lat, os_lon
df_neigh.loc['Lakeshore'] = ls_lat, ls_lon

In [64]:
sf_lat = 37.7792808
sf_lon = -122.4192363

map_sf = folium.Map(location=[sf_lat, sf_lon], zoom_start=12)

for neighborhood, coords in df_neigh.iterrows():
    lat, lon = coords[0], coords[1]
    
    label = neighborhood
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sf)  

In [65]:
map_sf

In [67]:
CLIENT_ID = 'VNBGQDHO2OBW0A4LRJJWRLKIFGQI3TF4P0JZMYZYT5TY0BZO'
CLIENT_SECRET = '2TVVXX51X31DSAKOBCIJLOXB4SWTSLY2Y5HOR44NJG2DLXXG'
CATEGORY_ID = '4d4b7105d754a06374d81259' # Restaurants
VERSION = '20180605'
LIM = 500

rad = 500

url_base = 'https://api.foursquare.com/v2/venues/explore?'

price_freq_df_rows = []

for neighborhood, coordinates in df_neigh.iterrows():
    lat, lon = coordinates[0], coordinates[1]
    
    price_buckets = []
    
    for price in range(1, 5):
        url = url_base + '&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&price={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lon, 
            rad,
            price,
            LIM,
            CATEGORY_ID)
        
        temp = requests.get(url, verify=False).json()['response']
        results = temp['groups'][0]['items']
        price_buckets.append(len(results))
        
    total = sum(price_buckets)
    if total > 0:
        temp = [i / total for i in price_buckets]
        price_buckets = temp
        
    price_freq_df_rows.append([neighborhood] + price_buckets)

/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:847

/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:847

/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:847

/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:847

/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:847

/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:847

In [68]:
cols = ['Neighborhood', '$', '$$', '$$$', '$$$$']
price_freq_df = pd.DataFrame(price_freq_df_rows, columns=cols).set_index('Neighborhood')

In [69]:
df_neigh_price = df_neigh.merge(price_freq_df, on='Neighborhood')
df_neigh_price

,Latitude,Longitude,$,$$,$$$,$$$$
Neighborhood,,,,,,
Bayview,37.728889,-122.392500,0.736842,0.210526,0.000000,0.052632
Bernal Heights,37.742986,-122.415804,0.416667,0.520833,0.062500,0.000000
Castro,37.760856,-122.434957,0.400000,0.500000,0.083333,0.016667
Chinatown,37.794301,-122.406376,0.441176,0.372549,0.156863,0.029412
Crocker Amazon,37.709378,-122.438587,0.000000,0.000000,0.000000,0.000000
Diamond Heights,37.745764,-122.441638,0.428571,0.428571,0.142857,0.000000
Civic Center,37.779594,-122.416794,0.558140,0.372093,0.069767,0.000000
Excelsior,37.721794,-122.435382,0.681818,0.295455,0.022727,0.000000
Financial District,37.793647,-122.398938,0.410959,0.383562,0.164384,0.041096


In [70]:
# set number of clusters
k = 4

sf_clustering = df_neigh_price[['$', '$$', '$$$', '$$$$']]

# run k-means clustering
kmeans = KMeans(n_clusters=k).fit(sf_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 0, 0, 2, 0, 3, 1, 0, 3], dtype=int32)

In [74]:
df_neigh_price.insert(len(df_neigh_price.columns), 'Cluster Label', kmeans.labels_)
df_neigh_price

,Latitude,Longitude,$,$$,$$$,$$$$,Cluster Label
Neighborhood,,,,,,,
Bayview,37.728889,-122.392500,0.736842,0.210526,0.000000,0.052632,1
Bernal Heights,37.742986,-122.415804,0.416667,0.520833,0.062500,0.000000,0
Castro,37.760856,-122.434957,0.400000,0.500000,0.083333,0.016667,0
Chinatown,37.794301,-122.406376,0.441176,0.372549,0.156863,0.029412,0
Crocker Amazon,37.709378,-122.438587,0.000000,0.000000,0.000000,0.000000,2
Diamond Heights,37.745764,-122.441638,0.428571,0.428571,0.142857,0.000000,0
Civic Center,37.779594,-122.416794,0.558140,0.372093,0.069767,0.000000,3
Excelsior,37.721794,-122.435382,0.681818,0.295455,0.022727,0.000000,1
Financial District,37.793647,-122.398938,0.410959,0.383562,0.164384,0.041096,0


In [75]:
map_clusters = folium.Map(location=[sf_lat, sf_lon], zoom_start=12)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for index, row in neigh_price_df.iterrows():
    lat, lon = row['Latitude'], row['Longitude']
    poi = index
    cluster = int(row['Cluster Label'])
    
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

In [76]:
df_neigh_price.reset_index().groupby('Cluster Label').mean()

,Latitude,Longitude,$,$$,$$$,$$$$
Cluster Label,,,,,,
0,37.777084,-122.425055,0.366241,0.476855,0.124213,0.032691
1,37.281693,-121.914457,0.763469,0.225486,0.004027,0.007018
2,37.748959,-122.462752,0.000000,0.000000,0.000000,0.000000
3,37.766630,-122.453673,0.529743,0.444767,0.022460,0.003030
